In [2]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [517 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,017 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,287 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,264 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,16

In [3]:
!apt-get -y install nvidia-cuda-toolkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libaccinj64-11.5 libatk-wrapper-java
  libatk-wrapper-java-jni libbabeltrace1 libcub-dev libcublas11 libcublaslt11
  libcudart11.0 libcufft10 libcufftw10 libcuinj64-11.5 libcupti-dev
  libcupti-doc libcupti11.5 libcurand10 libcusolver11 libcusolvermg11
  libcusparse11 libdebuginfod-common libdebuginfod1 libegl-dev libfontenc1
  libgail-common libgail18 libgl-dev libgl1-mesa-dev libgles-dev libgles1
  libglvnd-core-dev libglvnd-dev libglx-dev libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libipt2 libnppc11 libnppial11 libnppicc11 libnppidei11
  libnppif11 libnppig11 libnppim11 libnppist11 libnppisu11 libnppitc11
  libnpps11 libnvblas11 libnvidia-compute-495 libnvidia-compute-510
  libnvidia-compute-525 libnvidia-ml-dev libnvjpeg11 libnvrtc-builtins11.5
  libnvrtc11.2 libnvtoolsext1 libnvvm4 libopengl-de

In [4]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-qi4gkah0
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-qi4gkah0
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=758d6d3d966031f7f01d319d052f4a3c96c23ac9c5085a7217eac9122661aebd
  Stored in directory: /tmp/pip-ephem-wheel-cache-bnpddukc/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [16]:
from PIL import Image
import struct
import ctypes

img = Image.open('view.jpg')
# img = img.resize((5000, 5000))
(w, h) = img.size[0:2]
pix = img.load()
buff = ctypes.create_string_buffer(4 * w * h)
offset = 0
for j in range(h):
	for i in range(w):
		r = bytes((pix[i, j][0],))
		g = bytes((pix[i, j][1],))
		b = bytes((pix[i, j][2],))
		a = bytes((255,))
		struct.pack_into('cccc', buff, offset, r, g, b, a)
		offset += 4
out = open('in.data', 'wb')
out.write(struct.pack('ii', w, h))
out.write(buff.raw)
out.close()

In [56]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define CSC(call)  													\
do {																\
	cudaError_t res = call;											\
	if (res != cudaSuccess) {										\
		fprintf(stderr, "ERROR in %s:%d. Message: %s\n",			\
				__FILE__, __LINE__, cudaGetErrorString(res));		\
		exit(0);													\
	}																\
} while(0)


// текстурная ссылка <тип элементов, размерность, режим нормализации>
texture<uchar4, 2, cudaReadModeElementType> tex;


__device__ double rgb_to_luma(uchar4 p) {
  return 0.299 * p.x + 0.587 * p.y + 0.114 * p.z;
}


__global__ void kernel(uchar4* out_arr, int w, int h) {
	int idx = blockDim.x * blockIdx.x + threadIdx.x;
	int idy = blockDim.y * blockIdx.y + threadIdx.y;
	int offsetx = blockDim.x * gridDim.x;
	int offsety = blockDim.y * gridDim.y;

	for(int y = idy; y < h; y += offsety) {
	    for(int x = idx; x < w; x += offsetx) {
          double w11 = rgb_to_luma(tex2D(tex, x - 1, y - 1));
          double w12 = rgb_to_luma(tex2D(tex, x - 1, y));
          double w13 = rgb_to_luma(tex2D(tex, x - 1, y + 1));
          double w21 = rgb_to_luma(tex2D(tex, x, y - 1));
          double w22 = rgb_to_luma(tex2D(tex, x, y));
          double w23 = rgb_to_luma(tex2D(tex, x, y + 1));
          double w31 = rgb_to_luma(tex2D(tex, x + 1, y - 1));
          double w32 = rgb_to_luma(tex2D(tex, x + 1, y));
          double w33 = rgb_to_luma(tex2D(tex, x + 1, y + 1));

          double Gx = w13 + 2 * w23 + w33 - w11 - 2 * w21 - w31;
          double Gy = w31 + 2 * w32 + w33 - w11 - 2 * w12 - w13;
          int G = min(255, (int)sqrt(Gx * Gx + Gy * Gy));

          out_arr[y * w + x] = make_uchar4(G, G, G, 255);
      }
  }
}


int main() {
	int w, h;
	FILE* fp = fopen("in.data", "rb");
	fread(&w, sizeof(int), 1, fp);
	fread(&h, sizeof(int), 1, fp);
	uchar4* data = (uchar4*)malloc(sizeof(uchar4) * w * h);
	fread(data, sizeof(uchar4), w * h, fp);
	fclose(fp);

	// Подготовка данных для текстуры
	cudaArray *arr;
	cudaChannelFormatDesc ch = cudaCreateChannelDesc<uchar4>();
	CSC(cudaMallocArray(&arr, &ch, w, h));

	CSC(cudaMemcpyToArray(arr, 0, 0, data, sizeof(uchar4) * w * h, cudaMemcpyHostToDevice));

	// Подготовка текстурной ссылки, настройка интерфейса работы с данными
	tex.addressMode[0] = cudaAddressModeClamp;	// Политика обработки выхода за границы по каждому измерению
	tex.addressMode[1] = cudaAddressModeClamp;
	tex.channelDesc = ch;
	tex.filterMode = cudaFilterModePoint;		// Без интерполяции при обращении по дробным координатам
	tex.normalized = false;						// Режим нормализации координат: без нормализации

	// Связываем интерфейс с данными
	CSC(cudaBindTextureToArray(tex, arr, ch));

	uchar4* device_out;
	CSC(cudaMalloc(&device_out, sizeof(uchar4) * w * h));

	cudaEvent_t start, stop;
	CSC(cudaEventCreate(&start));
	CSC(cudaEventCreate(&stop));
	CSC(cudaEventRecord(start));

	kernel<<<dim3(1, 1), dim3(32, 32)>>>(device_out, w, h);
	CSC(cudaGetLastError());

	CSC(cudaEventRecord(stop));
	CSC(cudaEventSynchronize(stop));
	float t;
	CSC(cudaEventElapsedTime(&t, start, stop));
	CSC(cudaEventDestroy(start));
	CSC(cudaEventDestroy(stop));

	printf("time = %f ms\n", t);

	CSC(cudaMemcpy(data, device_out, sizeof(uchar4) * w * h, cudaMemcpyDeviceToHost));

	// Отвязываем данные от текстурной ссылки
	CSC(cudaUnbindTexture(tex));

	CSC(cudaFreeArray(arr));
	CSC(cudaFree(device_out));

	fp = fopen("out.data", "wb");
	fwrite(&w, sizeof(int), 1, fp);
	fwrite(&h, sizeof(int), 1, fp);
	fwrite(data, sizeof(uchar4), w * h, fp);
	fclose(fp);

	free(data);
	return 0;
}

time = 73.058434 ms



In [27]:
from PIL import Image
import struct
import ctypes

fin = open('out.data', 'rb')
(w, h) = struct.unpack('hi', fin.read(8))
buff = ctypes.create_string_buffer(4 * w * h)
fin.readinto(buff)
fin.close()
img = Image.new('RGBA', (w, h))
pix = img.load()
offset = 0
for j in range(h):
	for i in range(w):
		(r, g, b, a) = struct.unpack_from('cccc', buff, offset)
		pix[i, j] = (ord(r), ord(g), ord(b), ord(a))
		offset += 4
img.save('view.png')

In [17]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

double rgb_to_luma(uchar4 p) {
  return 0.299 * p.x + 0.587 * p.y + 0.114 * p.z;
}


void kernel(uchar4 *out, uchar4 *in, int w, int h) {

    for (int y = 0; y < h; y++) {
        for (int x = 0; x < w; x++) {
            double w11 = rgb_to_luma(in[max(min(y-1,h-1),0)*w + max(min(x-1,w-1),0)]);
            double w12 = rgb_to_luma(in[max(min(y,h-1),0)*w + max(min(x-1,w-1),0)]);
            double w13 = rgb_to_luma(in[max(min(y+1,h-1),0)*w + max(min(x-1,w-1),0)]);
            double w21 = rgb_to_luma(in[max(min(y-1,h-1),0)*w + max(min(x,w-1),0)]);
            double w22 = rgb_to_luma(in[max(min(y,h-1),0)*w + max(min(x,w-1),0)]);
            double w23 = rgb_to_luma(in[max(min(y+1,h-1),0)*w + max(min(x,w-1),0)]);
            double w31 = rgb_to_luma(in[max(min(y-1,h-1),0)*w + max(min(x+1,w-1),0)]);
            double w32 = rgb_to_luma(in[max(min(y,h-1),0)*w + max(min(x+1,w-1),0)]);
            double w33 = rgb_to_luma(in[max(min(y+1,h-1),0)*w + max(min(x+1,w-1),0)]);

            double gx = w13 + 2*w23 + w33 - w11 - 2*w21 - w31;
            double gy = w31 + 2*w32 + w33 - w11 - 2*w12 - w13;

            int grad = (int)sqrt(gx*gx + gy*gy);
            int result = min(255, grad);

            out[y * w + x] = make_uchar4(result, result, result, 0);
        }
    }
}


int main() {

	int w, h;
	FILE* fp = fopen("in.data", "rb");
	fread(&w, sizeof(int), 1, fp);
	fread(&h, sizeof(int), 1, fp);
	uchar4* data = (uchar4*)malloc(sizeof(uchar4) * w * h);
  uchar4* out = (uchar4*)malloc(sizeof(uchar4) * w * h);
	fread(data, sizeof(uchar4), w * h, fp);
	fclose(fp);

  clock_t begin = clock();
	kernel(out, data, w, h);
	printf("time = %f ms\n", ((double)(clock() - begin) / CLOCKS_PER_SEC) * 1000);


	fp = fopen("out.data", "wb");
	fwrite(&w, sizeof(int), 1, fp);
	fwrite(&h, sizeof(int), 1, fp);
	fwrite(data, sizeof(uchar4), w * h, fp);
	fclose(fp);

	free(data);

	return 0;
}

time = 5530.973000 ms

